In [2]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
train = pd.read_csv('data/train.csv')
print("Training Set:"% train.columns, train.shape, len(train))
test = pd.read_csv('data/test.csv')
print("Test Set:"% test.columns, test.shape, len(test))

Training Set: (31962, 3) 31962
Test Set: (17197, 2) 17197


In [4]:

import re
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")
print(test_clean)

          id                                              tweet
0      31963  studiolife aislife requires passion dedication...
1      31964    white supremacists want everyone to see the ...
2      31965  safe ways to heal your acne    altwaystoheal h...
3      31966  is the hp and the cursed child book up for res...
4      31967    3rd bihday to my amazing hilarious nephew el...
...      ...                                                ...
17192  49155  thought factory leftright polarisation trump u...
17193  49156  feeling like a mermaid  hairflip neverready fo...
17194  49157  hillary campaigned today in ohioomg amp used w...
17195  49158  happy at work conference right mindset leads t...
17196  49159  my   song so glad free download  shoegaze newm...

[17197 rows x 2 columns]


In [ ]:
%pip install scikit-learn

In [6]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=len(train_majority),   
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

label
1    29720
0    29720
Name: count, dtype: int64

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),])

In [8]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],
                                                    train_upsampled['label'],random_state = 0)

In [9]:
print(X_test)
print(X_test.sample())

1095                     bihday brother cool thebomb      
9766     gf asked me to make an account i told her i di...
26909         sunday   withyou happiness family  vinallop 
1386     been feeling low for ages and when the one per...
4616     chaplin  the dictator speech  via   theresista...
                               ...                        
17825    im looking forward to a few titles being annou...
30707    lol these pasty cakes telling the rockette to ...
15560    gorgeous evening family   lakesimcoe  willow b...
30566    rain or shine this treasure hunt is gonna go d...
9278     woman catcalled walking in newyork shuts down ...
Name: tweet, Length: 14860, dtype: object
23421    woo i graduated and got my drivers license all...
Name: tweet, dtype: object


In [10]:
import pickle
model = pipeline_sgd.fit(X_train, y_train)
# o_file = open('model/model.pickle',"wb")
# pickle.dump(model, o_file)
# y_predict = model.predict(X_test)
y_predict = model.predict(X_test.sample())
print(y_predict)
print(len(y_predict))
print(len(X_train))
# o_file.close()
# from sklearn.metrics import f1_score
# f1_score(y_test, y_predict)

[0]
1
44580
